In [55]:
import pandas as pd
import datetime
import time

import sys
sys.path.append('..')
from utils.utils_iFEMG_feature import read_label, df_save_csv
from utils.utils_signal_preprocess import df_time2stamp

# muscle name VS ch nums
- ch1: RF, Rectus femoris, 股直肌
- ch2: LF, Lateral femoris, 股外侧肌
- ch3: SarT, sartorius, 缝匠肌
- ch4: BF, Biceps femoris, 股二头肌
- ch5: SemiT, semitendinosus, 半腱肌
- ch6: VM, Vastus medialis, 股内侧肌

In [56]:
# 文件名及其对应的key
file_name_dict = {'aqian0':['affected', 'qian', '0'],
                  'aqian1_5':['affected', 'qian', '1.5'],
                  'aqian3':['affected', 'qian', '3'],
                  'ahou0':['affected', 'hou', '0'],
                  'ahou1_5':['affected', 'hou', '1.5'],
                  'ahou3':['affected', 'hou', '3'],
                  'hqian0':['healthy', 'qian', '0'],
                  'hqian1_5':['healthy', 'qian', '1.5'],
                  'hqian3':['healthy', 'qian', '3'],
                  'hhou0':['healthy', 'hou', '0'],
                  'hhou1_5':['healthy', 'hou', '1.5'],
                  'hhou3':['healthy', 'hou', '3']
                  }
# 列名与肌肉名称，仅在传感器通道与上述肌肉位置不对应时需要更改
b1_col_names_lst = ['time', 'FMG_RF', 'FMG_LF', 'FMG_SarT', 'FMG_BF', 'sEMG_RF', 'sEMG_LF', 'sEMG_SarT', 'sEMG_BF']
b2_col_names_lst = ['time', 'FMG_SemiT', 'FMG_VM', 'sEMG_SemiT', 'sEMG_VM']

In [57]:
# 读取数据
# lab path
# folder_path = r"E:\fuckinggggg\paper_iFEMG_4th\data"
# label_data = read_label(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.txt')
# b1_data = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b1" + '\\' + 'aqian0.db', header=None, sep=';')
# b2_data = pd.read_table(r"E:\fuckinggggg\paper_iFEMG_4th\data\241114s1\b2" + '\\' + 'aqian0.db', header=None, sep=';')
# desk path
b1_df = pd.read_table(r"D:\data\301\241114s1\b1\aqian0.db", sep=';', header=None)
b2_df = pd.read_table(r"D:\data\301\241114s1\b2\aqian0.db", sep=';', header=None)
label_lst = read_label(r"D:\data\301\241114s1\b1\aqian0.txt")

# 更改数据列名
b1_df = b1_df.drop(columns=9)
b2_df = b2_df.drop(columns=[3,4,7,8,9])
b1_df.columns = ['b1_time', 'FMG_RF', 'FMG_SarT', 'FMG_LF', 'FMG_SemiT', 'sEMG_RF', 'sEMG_SarT','sEMG_LF', 'sEMG_SemiT']
b2_df.columns = ['b2_time', 'FMG_BF', 'FMG_VM', 'sEMG_BF', 'sEMG_VM']

In [58]:
def addSuffix(df:pd.DataFrame, col_name:str)->pd.DataFrame:
    '''给df时间序列中的重复值添加后缀使其唯一，从而方便进行两个df按时间序列的merge，添加新列suffix
    
    Args:
    ----
        * `df`:
        * `col_name`:
    '''
    # 将时间列转换为指定格式的 datetime 类型
    df[col_name] = pd.to_datetime(df[col_name], format="%Y-%m-%d %H:%M:%S,%f")

    # 添加后缀处理重复值
    df['unique_time'] = df.groupby(col_name).cumcount() + 1
    df['suffix'] = df[col_name].astype(str) + '_' + df['unique_time'].astype(str)

    # 删除辅助列
    df.drop(columns=['unique_time'], inplace=True)
    return df

In [59]:
b1_df = addSuffix(b1_df, 'b1_time')
b2_df = addSuffix(b2_df, 'b2_time')

In [60]:
merged_df = pd.merge(b1_df, b2_df, how='outer', on='suffix')
merged_df

,b1_time,FMG_RF,FMG_SarT,FMG_LF,FMG_SemiT,sEMG_RF,sEMG_SarT,sEMG_LF,sEMG_SemiT,suffix,b2_time,FMG_BF,FMG_VM,sEMG_BF,sEMG_VM
0,2024-11-14 15:32:46.708,2991.0,0.0,88.0,0.0,1508.0,289.0,-891.0,721.0,2024-11-14 15:32:46.708_1,NaT,NaN,NaN,NaN,NaN
1,2024-11-14 15:32:46.708,2991.0,1469.0,88.0,3238.0,1237.0,283.0,-664.0,1091.0,2024-11-14 15:32:46.708_2,NaT,NaN,NaN,NaN,NaN
2,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3238.0,1398.0,284.0,-637.0,898.0,2024-11-14 15:32:46.709_1,NaT,NaN,NaN,NaN,NaN
3,2024-11-14 15:32:46.709,2991.0,1469.0,88.0,3230.0,957.0,285.0,-730.0,635.0,2024-11-14 15:32:46.709_2,NaT,NaN,NaN,NaN,NaN
4,2024-11-14 15:32:46.713,2991.0,1469.0,88.0,3230.0,945.0,284.0,-333.0,827.0,2024-11-14 15:32:46.713_1,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342524,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.597_1,2024-11-14 15:35:08.597,3203.0,3823.0,157.0,148.0
342525,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.597_2,2024-11-14 15:35:08.597,3203.0,3814.0,-189.0,61.0
342526,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.597_3,2024-11-14 15:35:08.597,3203.0,3814.0,-65.0,516.0
342527,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 15:35:08.597_4,2024-11-14 15:35:08.597,3203.0,3814.0,139.0,981.0


In [61]:
label_data = read_label(r"D:\data\301\241114s1\b1\aqian0.txt")
label_t_stamp = [] # 保存label文件中时间对应的时间戳，精度ms
for x in label_data:
    t = x[0] + " " + x[1]
    t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S.%f").timestamp()
    #ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
    label_t_stamp.append(t_array)

label_t_stamp

[1731569579.343986,
 1731569584.369697,
 1731569589.364151,
 1731569594.405721,
 1731569599.40898,
 1731569604.447724,
 1731569609.454855,
 1731569614.504423,
 1731569619.555754,
 1731569624.603994,
 1731569629.623484,
 1731569634.661786,
 1731569639.65352,
 1731569644.671694,
 1731569649.647895,
 1731569654.673263,
 1731569659.660728,
 1731569664.689167,
 1731569669.677253,
 1731569674.69825,
 1731569679.685412,
 1731569684.700191]

In [ ]:
for i in range(len(merged_df)):
    db_time = merged_df['b1_time'][i] if merged_df['b1_time'][i] is not pd.NaT else merged_df['b2_time'][i]
    db_stamp = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S,%f").timestamp()
    if fuck is pd.NaT:
        print('fuck you')

In [37]:
merged_df.to_csv(r"D:\data\301\241114s1\test.csv",index=False, date_format="%Y-%m-%d %H:%M:%S,%f")

In [36]:
df_save_csv(merged_df, r"D:\data\301\241114s1\test.csv")

File D:\data\301\241114s1\test.csv saved!


In [ ]:
def alienMerge(b1_file: str, b2_file: str, label_file: str)->pd.DataFrame:
    '''
    将单次测试的数据通过时间戳对应，生成dataframe
    '''
    # 读取文件
    try:
        label_data = read_label(label_file)
        b1_data = pd.read_table(b1_file, header=None, sep=';')
        b2_data = pd.read_table(b2_file, header=None, sep=';')
    except FileNotFoundError:
        print(f"file {b1_file} not found")
        return 0
    # 转换时间戳
    
    return

In [15]:
label_data

[['2024-11-14', '15:32:59.343986', '收缩'],
 ['2024-11-14', '15:33:04.369697', '舒张'],
 ['2024-11-14', '15:33:09.364151', '收缩'],
 ['2024-11-14', '15:33:14.405721', '舒张'],
 ['2024-11-14', '15:33:19.408980', '收缩'],
 ['2024-11-14', '15:33:24.447724', '舒张'],
 ['2024-11-14', '15:33:29.454855', '收缩'],
 ['2024-11-14', '15:33:34.504423', '舒张'],
 ['2024-11-14', '15:33:39.555754', '收缩'],
 ['2024-11-14', '15:33:44.603994', '舒张'],
 ['2024-11-14', '15:33:49.623484', '收缩'],
 ['2024-11-14', '15:33:54.661786', '舒张'],
 ['2024-11-14', '15:33:59.653520', '收缩'],
 ['2024-11-14', '15:34:04.671694', '舒张'],
 ['2024-11-14', '15:34:09.647895', '收缩'],
 ['2024-11-14', '15:34:14.673263', '舒张'],
 ['2024-11-14', '15:34:19.660728', '收缩'],
 ['2024-11-14', '15:34:24.689167', '舒张'],
 ['2024-11-14', '15:34:29.677253', '收缩'],
 ['2024-11-14', '15:34:34.698250', '舒张'],
 ['2024-11-14', '15:34:39.685412', '收缩'],
 ['2024-11-14', '15:34:44.700191', '舒张']]

In [36]:
datetime.datetime.now().timestamp()

1733542594.398275